# Data Cleaning Listado de Listado de Beneficiarios 2019-2022

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. Beneficiarios2019_2022 -> Dataset con el listado de TODOS los beneficiarios.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='utf-8', skiprows=0) for file in csv_files]

    # Print the number of rows for each DataFrame
    for i, df in enumerate(dataframes):
        print(f"Number of rows in DataFrame {i+1}: {df.shape[0]}")

    # Calculate the sum of rows in each individual dataset
    individual_row_sum = sum([df.shape[0] for df in dataframes])

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df, individual_row_sum


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2019-2022

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_19 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/fertilizantes_2019.csv', encoding='utf-8', skiprows=0)
beneficiarios_20 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2020.csv', encoding='utf-8', skiprows=0)
beneficiarios_21 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2021.csv', encoding='utf-8', skiprows=0)
beneficiarios_22 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2022.csv', encoding='utf-8', skiprows=0)

In [4]:
# Obtenemos las localidades únicas en el dataset.
Localidades_19 = beneficiarios_19[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_19 = Localidades_19.drop_duplicates()

Localidades_20 = beneficiarios_20[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_20 = Localidades_20.drop_duplicates()

Localidades_21 = beneficiarios_21[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_21 = Localidades_21.drop_duplicates()

Localidades_22 = beneficiarios_22[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_22 = Localidades_22.drop_duplicates()

In [5]:
Localidades_19['ENTIDAD_c_benef'] = Localidades_19['ENTIDAD'].apply(clean_text)
Localidades_19['MUNICIPIO_c_benef'] = Localidades_19['MUNICIPIO'].apply(clean_text)
Localidades_19['LOCALIDAD_c_benef'] = Localidades_19['LOCALIDAD'].apply(clean_text)

Localidades_20['ENTIDAD_c_benef'] = Localidades_20['ENTIDAD'].apply(clean_text)
Localidades_20['MUNICIPIO_c_benef'] = Localidades_20['MUNICIPIO'].apply(clean_text)
Localidades_20['LOCALIDAD_c_benef'] = Localidades_20['LOCALIDAD'].apply(clean_text)

Localidades_21['ENTIDAD_c_benef'] = Localidades_21['ENTIDAD'].apply(clean_text)
Localidades_21['MUNICIPIO_c_benef'] = Localidades_21['MUNICIPIO'].apply(clean_text)
Localidades_21['LOCALIDAD_c_benef'] = Localidades_21['LOCALIDAD'].apply(clean_text)

Localidades_22['ENTIDAD_c_benef'] = Localidades_22['ENTIDAD'].apply(clean_text)
Localidades_22['MUNICIPIO_c_benef'] = Localidades_22['MUNICIPIO'].apply(clean_text)
Localidades_22['LOCALIDAD_c_benef'] = Localidades_22['LOCALIDAD'].apply(clean_text)

In [6]:
# Creamos las dos keys de beneficiarios 2019
Localidades_19['KEY_benef_mun'] = Localidades_19['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_19[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_19['KEY_benef_loc'] = Localidades_19['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_19[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_19['LOCALIDAD_c_benef'].astype(str)


# Creamos las dos keys de beneficiarios 2020
Localidades_20['KEY_benef_mun'] = Localidades_20['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_20[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_20['KEY_benef_loc'] = Localidades_20['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_20[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_20['LOCALIDAD_c_benef'].astype(str)


# Creamos las dos keys de beneficiarios 2021
Localidades_21['KEY_benef_mun'] = Localidades_21['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_21[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_21['KEY_benef_loc'] = Localidades_21['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_21[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_21['LOCALIDAD_c_benef'].astype(str)


# Creamos las dos keys de beneficiarios 2022
Localidades_22['KEY_benef_mun'] = Localidades_22['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_22[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_22['KEY_benef_loc'] = Localidades_22['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_22[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_22['LOCALIDAD_c_benef'].astype(str)

In [7]:
Localidades_19.shape

(4578, 8)

In [8]:
Localidades_20.shape

(5383, 8)

In [9]:
Localidades_21.shape

(5915, 8)

In [10]:
Localidades_22.shape

(12514, 8)

In [11]:
# Obtenemos las localidades únicas en el dataset.
Municipios_19 = beneficiarios_19[['ENTIDAD', 'MUNICIPIO']]
Municipios_19 = Municipios_19.drop_duplicates()
# Obtenemos las localidades únicas en el dataset.
Municipios_20 = beneficiarios_20[['ENTIDAD', 'MUNICIPIO']]
Municipios_20 = Municipios_20.drop_duplicates()
# Obtenemos las localidades únicas en el dataset.
Municipios_21 = beneficiarios_21[['ENTIDAD', 'MUNICIPIO']]
Municipios_21 = Municipios_21.drop_duplicates()
# Obtenemos las localidades únicas en el dataset.
Municipios_22 = beneficiarios_22[['ENTIDAD', 'MUNICIPIO']]
Municipios_22 = Municipios_22.drop_duplicates()

In [12]:
# Estandarizamos la limpieza de los datos
Municipios_19['ENTIDAD_c_benef'] = Municipios_19['ENTIDAD'].apply(clean_text)
Municipios_19['MUNICIPIO_c_benef'] = Municipios_19['MUNICIPIO'].apply(clean_text)

# Estandarizamos la limpieza de los datos
Municipios_20['ENTIDAD_c_benef'] = Municipios_20['ENTIDAD'].apply(clean_text)
Municipios_20['MUNICIPIO_c_benef'] = Municipios_20['MUNICIPIO'].apply(clean_text)

# Estandarizamos la limpieza de los datos
Municipios_21['ENTIDAD_c_benef'] = Municipios_21['ENTIDAD'].apply(clean_text)
Municipios_21['MUNICIPIO_c_benef'] = Municipios_21['MUNICIPIO'].apply(clean_text)

# Estandarizamos la limpieza de los datos
Municipios_22['ENTIDAD_c_benef'] = Municipios_22['ENTIDAD'].apply(clean_text)
Municipios_22['MUNICIPIO_c_benef'] = Municipios_22['MUNICIPIO'].apply(clean_text)

In [13]:
# Creamos las dos keys de beneficiarios 
Municipios_19['KEY_benef_mun'] = Municipios_19['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_19[
    'MUNICIPIO_c_benef'].astype(str)
# Creamos las dos keys de beneficiarios 
Municipios_20['KEY_benef_mun'] = Municipios_20['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_20[
    'MUNICIPIO_c_benef'].astype(str)
# Creamos las dos keys de beneficiarios 
Municipios_21['KEY_benef_mun'] = Municipios_21['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_21[
    'MUNICIPIO_c_benef'].astype(str)
# Creamos las dos keys de beneficiarios 
Municipios_22['KEY_benef_mun'] = Municipios_22['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_22[
    'MUNICIPIO_c_benef'].astype(str)

In [14]:
Municipios_19.shape

(83, 5)

In [15]:
Municipios_20.shape

(184, 5)

In [16]:
Municipios_21.shape

(249, 5)

In [17]:
Municipios_22.shape

(1030, 5)

### 1.2 INEGI 2019 - 2022

Obtener listado único de municipios y listado único de localidades de inegi de 2019 a 2022 con sus claves.

In [18]:
path_dataset_inegi_2019 = '../../data/dataset_inegi_clean_2019.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2019 = pd.read_csv(path_dataset_inegi_2019)

In [19]:
path_dataset_inegi_2020 = '../../data/dataset_inegi_clean_2020.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2020 = pd.read_csv(path_dataset_inegi_2020)

In [20]:
path_dataset_inegi_2021 = '../../data/dataset_inegi_clean_2021.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2021 = pd.read_csv(path_dataset_inegi_2021)

In [21]:
path_dataset_inegi_2022 = '../../data/dataset_inegi_clean_2022.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2022 = pd.read_csv(path_dataset_inegi_2022)

In [22]:
dataset_inegi_2019['KEY_inegi_municipio'] = dataset_inegi_2019['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2019[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2019['KEY_inegi_localidad'] = dataset_inegi_2019['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2019[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2019['Localidad_c_inegi'].astype(str) 


dataset_inegi_2020['KEY_inegi_municipio'] = dataset_inegi_2020['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2020[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2020['KEY_inegi_localidad'] = dataset_inegi_2020['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2020[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2020['Localidad_c_inegi'].astype(str) 


dataset_inegi_2021['KEY_inegi_municipio'] = dataset_inegi_2021['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2021[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2021['KEY_inegi_localidad'] = dataset_inegi_2021['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2021[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2021['Localidad_c_inegi'].astype(str) 


dataset_inegi_2022['KEY_inegi_municipio'] = dataset_inegi_2022['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2022[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2022['KEY_inegi_localidad'] = dataset_inegi_2022['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2022[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2022['Localidad_c_inegi'].astype(str) 

In [23]:
dataset_inegi_2022.head()

,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi,CVE_LOC,Localidad_inegi,Entidad_c_inegi,Municipio_c_inegi,Localidad_c_inegi,KEY_inegi_municipio,KEY_inegi_localidad
0,1,Aguascalientes,1,Aguascalientes,1,Aguascalientes,aguascalientes,aguascalientes,aguascalientes,aguascalientes-aguascalientes,aguascalientes-aguascalientes-aguascalientes
1,1,Aguascalientes,1,Aguascalientes,94,Granja Adelita,aguascalientes,aguascalientes,granja adelita,aguascalientes-aguascalientes,aguascalientes-aguascalientes-granja adelita
2,1,Aguascalientes,1,Aguascalientes,96,Agua Azul,aguascalientes,aguascalientes,agua azul,aguascalientes-aguascalientes,aguascalientes-aguascalientes-agua azul
3,1,Aguascalientes,1,Aguascalientes,100,Rancho Alegre,aguascalientes,aguascalientes,rancho alegre,aguascalientes-aguascalientes,aguascalientes-aguascalientes-rancho alegre
4,1,Aguascalientes,1,Aguascalientes,102,Los Arbolitos [Rancho],aguascalientes,aguascalientes,los arbolitos [rancho],aguascalientes-aguascalientes,aguascalientes-aguascalientes-los arbolitos [r...


## INEGI Municipios únicos para cada año.

In [24]:
INEGI_UNIQUEMUN_2019 = dataset_inegi_2019.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2019 = INEGI_UNIQUEMUN_2019.drop_duplicates()

In [25]:
INEGI_UNIQUEMUN_2019.shape

(2475, 7)

In [26]:
INEGI_UNIQUEMUN_2020 = dataset_inegi_2020.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2020 = INEGI_UNIQUEMUN_2020.drop_duplicates()

In [27]:
INEGI_UNIQUEMUN_2020.shape

(2475, 7)

In [28]:
INEGI_UNIQUEMUN_2021 = dataset_inegi_2021.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2021 = INEGI_UNIQUEMUN_2021.drop_duplicates()

In [29]:
INEGI_UNIQUEMUN_2021.shape

(2475, 7)

In [30]:
INEGI_UNIQUEMUN_2022 = dataset_inegi_2022.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2022 = INEGI_UNIQUEMUN_2022.drop_duplicates()

In [31]:
INEGI_UNIQUEMUN_2022.shape

(2475, 7)

## INEGI Localidades únicas para cada año.

In [32]:
INEGI_UNIQUELOC_2019 = dataset_inegi_2019

INEGI_UNIQUELOC_2019 = INEGI_UNIQUELOC_2019.drop_duplicates()

In [33]:
INEGI_UNIQUELOC_2019.shape

(300240, 11)

In [34]:
INEGI_UNIQUELOC_2020 = dataset_inegi_2020

INEGI_UNIQUELOC_2020 = INEGI_UNIQUELOC_2020.drop_duplicates()

In [35]:
INEGI_UNIQUELOC_2020.shape

(300240, 11)

In [36]:
INEGI_UNIQUELOC_2021 = dataset_inegi_2021

INEGI_UNIQUELOC_2021 = INEGI_UNIQUELOC_2021.drop_duplicates()

In [37]:
INEGI_UNIQUELOC_2021.shape

(300240, 11)

In [38]:
INEGI_UNIQUELOC_2022 = dataset_inegi_2022

INEGI_UNIQUELOC_2022 = INEGI_UNIQUELOC_2022.drop_duplicates()

In [39]:
INEGI_UNIQUELOC_2022.shape

(300240, 11)